In [ ]:
import numpy as np
from random import random
import pandas as pd
from sklearn.model_selection import train_test_split
import math

In [ ]:
class Neuron:
    
    def __init__(self, weights_count):
        self.weights = [ (random()*2-1) for _ in range(weights_count) ] 
#         self.weights = [ random() for _ in range(weights_count) ] 
        self.weights_count = weights_count
        self.bias    = random()
        self.output  = None
        self.error   = None
        
    def get_output(self, input_values):
        self.output = self.bias
        
        for index in range(len(self.weights)):
            
            self.output += self.weights[index] * input_values[index]
            
#         for weight, value in zip(self.weights, input_values):
#             self.output += weight * value
        return self.output
    
    def set_error(self, error):
        self.error = error

In [ ]:
class Layer: 
 
    def __init__(self, 
                 neurons_count, 
                 activation_function, 
                 derivative_function, 
                 weights_count):
        self.neurons_count = neurons_count
        self.activation_function = activation_function
        self.derivative_function = derivative_function
        self.neurons = [ Neuron(weights_count) for _ in range(neurons_count)]
        
    def activate(self, input_values):
        for neuron in self.neurons: 
#             print(neuron.bias)
#             print(neuron.get_output(input_values))
#             print()
            
            neuron.output = self.activation_function(neuron.get_output(input_values))
            
    def get_output(self):
        return [ neuron.output for neuron in self.neurons ]
    
    def update_layer_weights(self, input_values, learning_rate):
        for neuron in self.neurons: 
            for index in range(neuron.weights_count):
                neuron.weights[index] += learning_rate * neuron.error * input_values[index]
            neuron.bias += learning_rate * neuron.error
        

In [ ]:
class HiddenLayer(Layer):
    def __init__(self, neurons_count, 
                 activation_function, 
                 derivative_function, 
                 weights_count):
        Layer.__init__(self, neurons_count, activation_function, derivative_function, weights_count)
    
    def compute_error(self, next_layer):
        for index in range(self.neurons_count):
#             print('intra in asta')
            delta = 0.0 
            for neuron in next_layer.neurons:
                delta += neuron.weights[index] * neuron.error
            error = self.derivative_function(self.neurons[index].output) * delta
            self.neurons[index].set_error(error)

In [ ]:
class OutputLayer(Layer):
    def __init__(self, neurons_count, 
                 activation_function, 
                 derivative_function, 
                 weights_count):
        
        Layer.__init__(self, neurons_count, activation_function, derivative_function, weights_count)
    
    def compute_error(self, expected):
#         print('merge compute pe output')
        for index in range(len(self.neurons)):
            delta = expected[index] - self.neurons[index].output
            self.neurons[index].set_error(delta * self.derivative_function(self.neurons[index].output))
            
#         for value, neuron in zip(expected, self.neurons):
#             print('intra in astalalta')
#             delta = value - neuron.output
#             neuron.set_error(delta * self.derivative_function(neuron.output))
#             print(neuron.error)
#         for neuron in self.neurons:
#             print(neuron.error)

In [ ]:
class Network:
    
    def __init__(self,
                layer_count, 
                neuron_count, 
                activation_function, 
                derivative_function, 
                input_count, 
                class_count):
        self.layers = self.initialize_layers(layer_count, 
                                        neuron_count, 
                                        activation_function, 
                                        derivative_function, 
                                        input_count, 
                                        class_count)
        self.class_count = class_count
        self.learning_rate = 0.3
        
    def forward(self, input_values):
        inputs = input_values
        for layer in self.layers:
            layer.activate(inputs)
            inputs = layer.get_output()
        
    def backward(self, input_values, expected):
        self.layers[-1].compute_error(expected)
        self.layers[-1].update_layer_weights(input_values, self.learning_rate)
        
        for index in reversed(range(len(self.layers) - 1)):
            self.layers[index].compute_error(self.layers[index + 1])
        
        self.layers[0].update_layer_weights(input_values, self.learning_rate)
        
#         print('hello there')
        
        for index in range(1, len(self.layers)):
            self.layers[index].update_layer_weights(self.layers[index - 1].get_output(), self.learning_rate)
        
    def initialize_layers(self, layer_count, neuron_count, activation_function, derivative_function, input_count, class_count):
        layers = [ HiddenLayer(neuron_count, activation_function, derivative_function, input_count) ]
        for _ in range(layer_count - 1):
            layers.append(HiddenLayer(neuron_count, activation_function, derivative_function, len(layers[-1].neurons)))
        layers.append( OutputLayer(class_count, activation_function, derivative_function, len(layers[-1].neurons)) )
        
        return layers
        
        

In [ ]:
class Application:
    
    def __init__(self):
        self.class_count = 3  
        self.data = pd.read_csv("dataset.csv")
        self.columns = list(self.data)
        self.classes = self.columns.pop()
        self.normalize_data()
        self.train_data = []
        self.test_data  = []
        self.split_data()
        self.network = Network(
            neuron_count=15, 
            layer_count=2, 
            activation_function=lambda x: 1/(1 + math.exp(-x)), 
            derivative_function=lambda x: x * (1-x), 
            input_count=len(self.columns),
            class_count=self.class_count
        )
        
    def train(self):
        iteration_count = 10
        for _ in range(iteration_count):
            self.train_data = self.train_data.sample(frac=1)
            for _, example in self.train_data.iterrows():
                example = example.tolist()
                expected_class = example.pop()
                self.network.forward(example)
                expected = [ 1 if index == expected_class - 1 else 0 for index in range(self.class_count)]
                self.network.backward(example, expected)
#         print("hello")
                
    def test(self):
        correct_count = 0 
        for _, row in self.test_data.iterrows():
            row = row.tolist()
            expected_class = row.pop()
            predicted_class = self.classify(row) + 1
#             print(predicted_class)
#             print(expected_class)
#             print()
            if predicted_class == expected_class:
                correct_count += 1
                
        return float(correct_count) / float(len(self.test_data))
    
    def classify(self, input_values):
#             print(type(input_values))
            self.network.forward(input_values)
            output = self.network.layers[-1].get_output()
            max_index = np.argmax(output)
            return max_index
            
    def normalize_data(self):
        for c in self.columns:
            column = [float(item) for item in self.data[c]]
            minimum = min(column)
            maximum = max(column)
            for index in range(self.data.shape[0]):
                old_value = float(self.data.iloc[index][c])
                if (maximum == minimum):
                    new_value = 0.5
                else:
                    new_value = (old_value - minimum) / (maximum - minimum)
                self.data.ix[index, c] = new_value
                
    def split_data(self):
        self.train_data, self.test_data = train_test_split(self.data, test_size = 0.25, random_state=42)


In [ ]:
# data = pd.read_csv("dataset.csv")
# # data = data.sample(frac=1)
# for example in data.iterrows():
# #     print(type(example[1]))
#     print(example)
#     print(example[1].tolist())
#     break
    

In [ ]:
a = Application() 

In [ ]:
a.train()

In [ ]:
a.test()

In [ ]:
# a.train_data